#### Using custom operators in join conditions

Another use case for relationships is the use of custom operators, such as _PostgreSQL_'s __"is contained within"__ `<<` operator when _joining with types such as INET and CIDR_. For _custom boolean operators_ we use the `Operators.bool_op()` function:

```
inet_column.bool_op("<<")(cidr_column)
```

A comparison like the above may be __used directly__ with `relationship.primaryjoin` when _constructing_ a `relationship()`.

In [1]:
from sqlalchemy import Column, Integer
from sqlalchemy.orm import relationship
from sqlalchemy.dialects.postgresql import INET, CIDR
from sqlalchemy.ext.declarative import declarative_base

In [2]:
Base = declarative_base()

In [3]:
class IPA(Base):
    __tablename__ = "ip_address"
    
    id = Column(Integer, primary_key=True)
    v4address = Column(INET)
    
    network = relationship(
        "Network",
        primaryjoin="IPA.v4address.bool_op('<<')" "(foreign(Network.v4representation))",
        viewonly=True,
    )

In [4]:
class Network(Base):
    __tablename__ = "network"
    
    id = Column(Integer, primary_key=True)
    v4representation = Column(CIDR)

Above, a query such as:

```
session.query(IPA).join(IPA.network)
```

Will render as:

```
SELECT ip_address.id AS ip_address_id, ip_address.v4address AS ip_address_v4address
FROM ip_address JOIN network ON ip_address.v4address << network.v4representation
```